In [1]:
using Gen

# A binomial model: sum of Bernoullis

In [2]:
struct DiracDelta <: Gen.Distribution{Float64} end
const diracdelta = DiracDelta()

Gen.logpdf(diracdelta, x, u) = (x == u) ? 0.0 : -Inf
Gen.random(diracdelta, u) = u
diracdelta(u) = Gen.random(diracdelta, u)

DiracDelta()

In [3]:
@gen function bin_sim(N::Int64, p::Float64)
    x = zeros(N,1)
    for i=1:N
        x[i] = @trace(bernoulli(p), :x => i)
    end
    
    return sum(x)
end

DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[Int64, Float64], ##bin_sim#371, Bool[0, 0], false)

In [12]:
@gen function data_generator(M::Int64)
    
    N = @trace(geometric(0.025), :N)
    p = @trace(beta(5,5), :p)
    
    ys = zeros(M,1)
    
    for i=1:M
        ys[i] = @trace( diracdelta(bin_sim(N,p)), :y => i )
    end
    
    return ys
end     

DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[Int64], ##data_generator#374, Bool[0], false)

In [13]:
tr = simulate(data_generator, (10,));

In [14]:
println(tr[:N])
println(tr[:p])
ys = [tr[:y=>i] for i=1:10]

76
0.2645568164780811


10-element Array{Float64,1}:
 17.0
 19.0
 21.0
 19.0
 19.0
 22.0
 21.0
 21.0
 22.0
 22.0

In [15]:
tr

Gen.DynamicDSLTrace{DynamicDSLFunction{Any}}(DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[Int64], ##data_generator#374, Bool[0], false), Trie{Any,Gen.ChoiceOrCallRecord}(Dict{Any,Gen.ChoiceOrCallRecord}(:N => Gen.ChoiceOrCallRecord{Int64}(76, -5.613032860919967, NaN, true),:p => Gen.ChoiceOrCallRecord{Float64}(0.2645568164780811, -0.1022051180386061, NaN, true)), Dict{Any,Trie{Any,Gen.ChoiceOrCallRecord}}(:y => Trie{Any,Gen.ChoiceOrCallRecord}(Dict{Any,Gen.ChoiceOrCallRecord}(7 => Gen.ChoiceOrCallRecord{Float64}(21.0, 0.0, NaN, true),4 => Gen.ChoiceOrCallRecord{Float64}(19.0, 0.0, NaN, true),9 => Gen.ChoiceOrCallRecord{Float64}(22.0, 0.0, NaN, true),10 => Gen.ChoiceOrCallRecord{Float64}(22.0, 0.0, NaN, true),2 => Gen.ChoiceOrCallRecord{Float64}(19.0, 0.0, NaN, true),3 => Gen.ChoiceOrCallRecord{Float64}(21.0, 0.0, NaN, true),5 => Gen.ChoiceOrCallRecord{Float64}(19.0, 0.0, NaN, true),8 => Gen.ChoiceOrCallRecord{Float64}(21.0, 0.0, NaN, true),6 => Gen.ChoiceOrCallR

## Generate some data -- then we'll try  estimating parameters from it

In [ ]:
using Distributions

In [ ]:
y = rand(Binomial(10, 0.5), 100,1)

# Inference

## Importance Sampling

In [ ]:
function posterior_sample_importance(ys, n_samples::Int=1000)
    
    # Set the observed data
    observations = Gen.choicemap()
    for (i, y) in enumerate(ys)
        observations[(:y, i)] = y
    end
    
    (traces, log_ws, _) = Gen.importance_sampling(data_generator, (length(ys),), observations, n_samples)
    
    return traces, log_ws
end

In [ ]:
(trs, lws) = posterior_sample_importance(y, 1000)

In [ ]:
simulate(data_generator, (100,))

# An even dumber example: a Gaussian model

An improper distribution -- i.e., it's not correctly normalized. We'll see if we can sample from it anyway.

In [ ]:
struct DPMGaussian <: Gen.Distribution{Float64} end
const dpmgaussian = DPMGaussian()

Gen.random(dpmgaussian, mu, sigma) = mu
Gen.logpdf(dpmgaussian, x, mu, sigma) = -0.5*(x - mu)^2 / (sigma^2) - 0.5*log(2 * pi * sigma^2)

dpmgaussian(mu, sigma) = Gen.random(dpmgaussian, mu, sigma)

## Model the data as iid normal

In [ ]:
@gen function dpm_gaussian(N::Int64)
    mu = @trace(uniform_continuous(5, 12), :mu)
    sigma = @trace(gamma(1,1), :sigma)
    
    z = zeros(N,1)
    for i=1:N
        z[i] = @trace(dpmgaussian(mu, sigma), (:z, i))
    end
    
    return z
end

## Make some synthetic data to play with

In [ ]:
z = 7 .+ 3.14.*randn(5000,1);

## Infer a posterior distribution on mean and variance

### Importance sampling

In [ ]:
observations = Gen.choicemap()
for (i,v) in enumerate(z)
    observations[(:z, i)] = v
end

(trs, lws, _) = Gen.importance_sampling(dpm_gaussian, (length(z),), observations, 500);

In [ ]:
ws = map(exp, lws);

In [ ]:
mus = [tr[:mu] for tr in trs];
sigmas = [tr[:sigma] for tr in trs];

In [ ]:
expected_mu = sum(mus .* ws) / sum(ws)
expected_sigma = sum(sigmas .* ws) / sum(ws)

In [ ]:
expected_mu

### MCMC sampling

Define a proposal distribution

In [ ]:
function dumb_update(prev_trace,N)
    delta_mu = @trace(uniform_continuous(-1.0, 1.0), :dmu)
    new_sigma = @trace(gamma(1,1), :new_sigma)
    
    new_choices = Gen.choicemap()
    new_choices[:mu] = prev_trace[:mu] + delta_mu
    new_choices[:sigma] = new_sigma
    
    for i=1:N
        delta_x = @trace(uniform_continuous(-0.1, 1.0), (:dz, i))
        new_choices[(:z, i)] = prev_trace[(:z, i)] + delta_x
    end
    
    new_tr, _, _, _ = Gen.update(prev_trace, (), (), new_choices)
    return new_trace
end

In [ ]:
function mcmc_sample_gauss(n_posterior_samples, z)
    
    # Initialize the trace; load observations
    tr = simulate(dpm_gaussian, (length(z), ) )
    observations = Gen.choicemap()
    for i=1:length(z)
        observations[(:z, i)] = z[i]
    end
    Gen.update(tr, (length(z),), (), observations)
    
    # Store the quantities of interest
    mus = zeros(n_posterior_samples, 1)
    sigmas = zeros(n_posterior_samples, 1)
    
    # Take metropolis-hastings steps
    for i=1:n_posterior_samples
        tr, _ = Gen.mh(tr, dumb_proposal, (length(z),)) 
        mus[i] = tr[:mu]
        sigmas[i] = tr[:sigma]
    end
    
    return mus, sigmas
    
end 

In [ ]:
mus, sigmas = mcmc_sample_gauss(1000, z)

In [ ]:
Gen.select()